<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%207/homework_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# from nltk.corpus import conll2002
import nltk 
nltk.download('conll2002')
from nltk.corpus import conll2002
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support
import pickle

# Assignment 7: NER
# Rebecca Iglesias-Flores and Shubhankar Patankar
# This is just to help you get going. Feel free to
# add to or modify any part of it.

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


In [0]:
def hasDot(word):
  return int('.' in word)

def hasApost(word):
  return int('\'' in word)

def hasHyph(word):
  return int('-' in word)

def hasNN(pos):
  return int('NN' in pos)

def isCap(word):
    ## if first letter of the word is capitalized
  return int(word[0].isupper()) 

def hasCap(word):
    ## if any letter of the word is capitalized or not
  return int(word.islower())

def hasAcc(word):
  for char in word:
    if char in accents:
      return int(True)
  return int(False)

def hasDig(word):
  ## assigns a 1 if there's a digit in the word
  return int(word.isalpha())

def prefix(word):
  return word[:4]

def suffix(word):
  return word[-4:]

def getfeats(word, pos_tag, o):
  """ This takes the word in question and
  the offset with respect to the instance
  word """
  o = str(o)
  features = [
              (o + 'word', word),
              # TODO: add more features here.
              (o + 'hasDot', hasDot(word)),
              (o + 'hasApost', hasApost(word)),
              (o + 'hasHyph', hasHyph(word)),
              (o + 'hasNN', hasNN(pos_tag)),
              (o + 'isCap', isCap(word)),
              (o + 'hasCap', hasCap(word)),
              # (o + 'hasAcc', hasAcc(word)),
              (o + 'hasDig', hasDig(word)),
              (o + 'prefix', prefix(word)),
              (o + 'suffix', suffix(word))
              ]
  return features
    
def word2features(sent, i):
  """ The function generates all features
  for the word at position i in the
  sentence."""
  features = []
  # the window around the token
  for o in [-1,0,1]:
    if i+o >= 0 and i+o < len(sent):
      word = sent[i+o][0]
      pos_tag = sent[i+o][1]
      featlist = getfeats(word, pos_tag, o)
      features.extend(featlist)
  
  return dict(features)

In [0]:
# train_sents = list(conll2002.iob_sents('esp.train'))
# dev_sents = list(conll2002.iob_sents('esp.testa'))
# test_sents = list(conll2002.iob_sents('esp.testb'))

In [0]:
# train_feats = []
# train_labels = []

# for sent in train_sents:
#   for i in range(len(sent)):
#     feats = word2features(sent, i)
#     train_feats.append(feats)
#     train_labels.append(sent[i][-1])

# vectorizer = DictVectorizer()
# X_train = vectorizer.fit_transform(train_feats)

In [4]:
if __name__ == "__main__":

  train_sents = list(conll2002.iob_sents('esp.train'))
  train_feats = []
  train_labels = []
  for sent in train_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      train_feats.append(feats)
      train_labels.append(sent[i][-1])

  vectorizer = DictVectorizer()
  X_train = vectorizer.fit_transform(train_feats)

  model = Perceptron(verbose = 1)
  model.fit(X_train, train_labels)
  pickle.dump(model, open('drive/My Drive/CIS-530/Homework 7/Results/model', 'wb'))

  # TRAINING SET
  y_pred_train = model.predict(X_train)  
  j = 0
  print("Writing to train_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/train_results.txt' 
  with open(outfile_path, "w") as out:
    for sent in train_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_train[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word, gold, pred))
    out.write("\n")

  # DEVELOPMENT SET
  dev_sents = list(conll2002.iob_sents('esp.testa'))
  dev_feats = []
  dev_labels = []
  for sent in dev_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      dev_feats.append(feats)
      dev_labels.append(sent[i][-1])
  X_dev = vectorizer.transform(dev_feats)
  y_pred_dev = model.predict(X_dev)
  j = 0
  print("Writing to dev_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/dev_results.txt'
  with open(outfile_path, "w") as out:
    for sent in dev_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_dev[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word,gold,pred))
    out.write("\n")

  # TEST SET
  test_sents = list(conll2002.iob_sents('esp.testb'))
  test_feats = []
  test_labels = []
  for sent in test_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      test_feats.append(feats)
      test_labels.append(sent[i][-1])
  X_test = vectorizer.transform(test_feats)
  y_pred_test = model.predict(X_test)
  j = 0
  print("Writing to test_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/test_results.txt'
  with open(outfile_path, "w") as out:
    for sent in test_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_test[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word,gold,pred))
    out.write("\n")

  print("Now run: python conlleval.py test_results.txt")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 137.61, NNZs: 9027, Bias: -1.160000, T: 264715, Avg. loss: 0.076645
Total training time: 0.16 seconds.
-- Epoch 2
Norm: 181.63, NNZs: 11503, Bias: -1.530000, T: 529430, Avg. loss: 0.053102
Total training time: 0.30 seconds.
-- Epoch 3
Norm: 213.14, NNZs: 13020, Bias: -1.800000, T: 794145, Avg. loss: 0.046998
Total training time: 0.44 seconds.
-- Epoch 4
Norm: 237.92, NNZs: 14061, Bias: -2.000000, T: 1058860, Avg. loss: 0.043453
Total training time: 0.57 seconds.
-- Epoch 5
Norm: 257.73, NNZs: 14845, Bias: -2.110000, T: 1323575, Avg. loss: 0.040266
Total training time: 0.71 seconds.
-- Epoch 6
Norm: 274.95, NNZs: 15328, Bias: -2.200000, T: 1588290, Avg. loss: 0.037948
Total training time: 0.85 seconds.
-- Epoch 7
Norm: 289.40, NNZs: 15769, Bias: -2.310000, T: 1853005, Avg. loss: 0.036562
Total training time: 0.99 seconds.
-- Epoch 8
Norm: 303.54, NNZs: 16160, Bias: -2.420000, T: 2117720, Avg. loss: 0.037257
Total training time: 1.13 seconds.
-- Epoch 9
Norm: 316.86, NNZ

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   18.1s finished


Writing to train_results.txt
Writing to dev_results.txt
Writing to test_results.txt
Now run: python conlleval.py test_results.txt


In [5]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/train_results.txt'

processed 264715 tokens with 18797 phrases; found: 21168 phrases; correct: 16387.
accuracy:  98.52%; precision:  77.41%; recall:  87.18%; FB1:  82.01
              LOC: precision:  75.34%; recall:  91.55%; FB1:  82.66  5970
             MISC: precision:  61.25%; recall:  77.68%; FB1:  68.49  2756
              ORG: precision:  79.33%; recall:  80.51%; FB1:  79.92  7500
              PER: precision:  86.02%; recall:  98.38%; FB1:  91.78  4942


In [6]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/dev_results.txt'

processed 52923 tokens with 4351 phrases; found: 5397 phrases; correct: 2703.
accuracy:  93.99%; precision:  50.08%; recall:  62.12%; FB1:  55.46
              LOC: precision:  46.31%; recall:  75.30%; FB1:  57.35  1600
             MISC: precision:  27.53%; recall:  36.63%; FB1:  31.44  592
              ORG: precision:  57.74%; recall:  50.24%; FB1:  53.73  1479
              PER: precision:  54.75%; recall:  77.33%; FB1:  64.11  1726


In [7]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/test_results.txt'

processed 51533 tokens with 3558 phrases; found: 4417 phrases; correct: 2389.
accuracy:  95.55%; precision:  54.09%; recall:  67.14%; FB1:  59.91
              LOC: precision:  59.19%; recall:  71.31%; FB1:  64.69  1306
             MISC: precision:  25.33%; recall:  39.53%; FB1:  30.88  529
              ORG: precision:  59.67%; recall:  61.29%; FB1:  60.47  1438
              PER: precision:  54.55%; recall:  84.90%; FB1:  66.42  1144
